In [ ]:
!pip install lightfm

     |████████████████████████████████| 310 kB 30.4 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705354 sha256=ec88a68a7c9ad45435c7a18ee7c4708350e890dd565321051b8b641d0cefda97
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [ ]:
import pandas as pd
import numpy as np
from lightfm.data import Dataset
from scipy.io import mmwrite
from sklearn.model_selection import train_test_split
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score

In [ ]:
data_path = '/content/drive/MyDrive/K-Digital training/파이널 프로젝트/data/'
data = pd.read_csv(data_path + 'fp_train_dataset.csv')

In [ ]:
data

,label,user_id,imp_id,content_id,server_time_kst,content_bid_price,device_type,c_user_gender,c_user_age,user_following_count,user_pay_count,user_parcel_post_count,user_transfer_count,user_chat_count,advertiser_id,content_name,content_keyword,content_price,content_flag_used,c_content_category_id_1,c_content_category_id_2,c_content_category_id_3,content_emergency_count,content_comment_count,content_interest_count,content_favorite_count,advertiser_favorite_count,advertiser_grade,advertiser_item_count,advertiser_interest_count,advertiser_review_count,advertiser_comment_count,advertiser_follow_count,advertiser_pay_count,advertiser_parcel_post_count,advertiser_transfer_count,advertiser_chat_count,hour,view_count,content_count,count
0,0,11398944,97f9612d73d214f7a88e,137072639,2021-08-31 09:12:21.233000+09:00,70,a,1,14,1,0,0,0,0,12851816,#당일발송#아이폰8플러스 64G/256G 대량판매 등급별초특가!,"아이폰8플러스중고,아이폰8플러스",275000,1,600,600700,600700001,2,2,8157,307,2131,5285,247,4525,542,739,0,496,0,0,11,9,4,0.0,NaN
1,0,11501601,971a612d98320db51ae0,146010313,2021-08-31 11:47:21.476000+09:00,50,a,1,15,1,0,0,0,0,3947717,에어팟2세대 유선충전 미개봉 (애플코리아정품),"에어팟,에어팟미개봉,에어팟유선,에어팟2세대,무선에어팟",125000,2,600,600500,600500010,0,0,12449,402,7272,18135,906,40305,1930,408,0,83,8,0,14,11,9,0.5,NaN
2,0,1083083,97f9612e343117ec67ae,163137784,2021-08-31 22:53:02.887000+09:00,60,a,1,26,0,1,8,0,1,10197556,노트10플러스 256기가 화이트 S급 48272,"노트10플러스,갤럭시노트10플러스",450000,1,600,600700,600700001,0,0,46,0,5649,26531,595,11355,2661,1056,0,1526,0,11,1022,22,1,-0.3,NaN
3,0,8924184,9919612e13c623135015,163185401,2021-08-31 20:34:45.403000+09:00,55,a,2,47,2,0,0,0,0,4507787,람빠소 따자르 BG 엔틱 퀸 침대 크랙 상품,"침대,퀸침대,엔틱침대,엔틱가구,금장침대",700000,2,810,810100,810100200,0,0,129,7,146,30,124,3658,3,182,0,2,0,0,1,20,3,-0.1,0.0
4,0,899007,97ca612db02a12527ae8,160945625,2021-08-31 13:29:53.847000+09:00,55,a,0,0,7,0,39,0,0,74344938,[200%정품] 샤넬 핸들 장식의 미니 플랩백 블랙 금장 AS2431,"샤넬,샤넬가방,샤넬핸들장식플랩백,AS2431,샤넬코코핸들",5419000,2,430,430100,430100100,1,0,982,31,812,70,326,986,7,5,0,9,0,0,0,13,20,1.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575187,0,123120,99d1612dc92621f14bc3,155561048,2021-08-31 15:16:16.094000+09:00,50,a,2,51,34,0,0,0,0,87148,럭키슈에뜨 원피스 36 n5162 애슐리상점,"애슐리상점,럭키슈에뜨,원피스,36사이즈",120000,1,310,310120,310120020,0,0,981,11,2183,12120,7036,115655,1235,211,0,351,386,32,1,15,7,0.3,0.0
575188,0,4700908,99d1612ddb092235ee9d,123986789,2021-08-31 16:33:26.171000+09:00,50,a,1,63,12,0,0,0,0,4776936,[신품반값] 미니기타 절반가에 팝니다,"미니기타,미니,통기타,여행용,기타",150000,2,920,920200,920200200,1,0,3747,73,210,532,20,1053,54,47,0,115,0,0,8,16,4,0.0,0.0
575189,0,11609078,9919612db34721995743,161508998,2021-08-31 14:07:55.175000+09:00,50,a,1,65,1,0,1,0,0,8602749,기가골프 남자골프채풀세트 그라파이트 단조,"기가골프,남자골프채풀세트,포지드,그라파이트,단조",400000,1,700,700600,700600300,0,0,355,5,357,546,403,1507,58,124,0,130,0,2,11,14,1,-0.3,0.0
575190,0,1119616,971a612d0ffc0ca64a19,151760665,2021-08-31 02:06:10.703000+09:00,200,a,1,31,13,0,1,0,2,4565332,초강력 단기다이어트 지방분해 식욕억제 탄수화물차단 후덜덜라인,"후덜덜라인,동국제약,디톡스,다이어트,지방분해",13000,2,410,410900,410900,2,2,7253,723,5816,26955,281,245194,2784,1373,0,779,0,111,3,2,1,-0.3,0.0


In [ ]:
item_meta = data[['advertiser_id','advertiser_favorite_count','advertiser_grade','advertiser_interest_count']]

In [ ]:
item_meta

,advertiser_id,advertiser_favorite_count,advertiser_grade,advertiser_interest_count
0,12851816,2131,5285,4525
1,3947717,7272,18135,40305
2,10197556,5649,26531,11355
3,4507787,146,30,3658
4,74344938,812,70,986
...,...,...,...,...
575187,87148,2183,12120,115655
575188,4776936,210,532,1053
575189,8602749,357,546,1507
575190,4565332,5816,26955,245194


In [ ]:
item_meta['advertiser_favorite_count']

0         2131
1         7272
2         5649
3          146
4          812
          ... 
575187    2183
575188     210
575189     357
575190    5816
575191    2063
Name: advertiser_favorite_count, Length: 575192, dtype: int64

In [ ]:
test = item_meta[item_meta['advertiser_id'] == 12851816].values[0][3]
test

4525

In [ ]:
data.drop(columns=['label',
 'imp_id',
 'content_id',
 'server_time_kst',
 'content_bid_price',
 'device_type',
 'c_user_gender',
 'c_user_age',
 'user_following_count',
 'user_pay_count',
 'user_parcel_post_count',
 'user_transfer_count',
 'user_chat_count',
 'content_name',
 'content_keyword',
 'content_price',
 'content_flag_used',
 'c_content_category_id_1',
 'c_content_category_id_2',
 'c_content_category_id_3',
 'content_emergency_count',
 'content_comment_count',
 'content_interest_count',
 'content_favorite_count',
 'advertiser_favorite_count',
 'advertiser_grade',
 'advertiser_item_count',
 'advertiser_interest_count',
 'advertiser_review_count',
 'advertiser_comment_count',
 'advertiser_follow_count',
 'advertiser_pay_count',
 'advertiser_parcel_post_count',
 'advertiser_transfer_count',
 'advertiser_chat_count',
 'hour',
'view_count',
 'count'], inplace=True)

In [ ]:
data

,user_id,advertiser_id,content_count
0,11398944,12851816,0.0
1,11501601,3947717,0.5
2,1083083,10197556,-0.3
3,8924184,4507787,-0.1
4,899007,74344938,1.6
...,...,...,...
575187,123120,87148,0.3
575188,4700908,4776936,0.0
575189,11609078,8602749,-0.3
575190,1119616,4565332,-0.3


In [ ]:
ratings = data
ratings_source = [(ratings['user_id'][i], ratings['advertiser_id'][i]) for i in range(ratings.shape[0])]

In [ ]:
item_features_source = [(item_meta['advertiser_id'][i],
                        [item_meta['advertiser_favorite_count'][i],
                         item_meta['advertiser_grade'][i]]) for i in range(item_meta.shape[0])]

In [ ]:
dataset = Dataset()
dataset.fit(users=ratings['user_id'].unique(),
            items=ratings['advertiser_id'].unique(),
            item_features=item_meta[item_meta.columns[1:]].values.flatten()
            )

In [ ]:
interactions, weights = dataset.build_interactions(ratings_source)
item_features = dataset.build_item_features(item_features_source)

In [ ]:
# Split Train, Test data
# train, test = train_test_split(interactions, test_size=0.1)
train, test = cross_validation.random_train_test_split(interactions, test_percentage=0.1)
train, test = train.tocsr().tocoo(), test.tocsr().tocoo()
train_weights = train.multiply(weights).tocoo()

In [ ]:
from hyperopt import fmin, hp, tpe, Trials

# Define Search Space
trials = Trials()
space = [hp.choice('no_components', range(10, 50, 10)),
         hp.uniform('learning_rate', 0.01, 0.05)]

In [ ]:
# Define Objective Function
def objective(params):
    no_components, learning_rate = params

    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

    model.fit(interactions=train,
              item_features=item_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

    test_precision = precision_at_k(model, test, k=5, item_features=item_features).mean()
    print("no_comp: {}, lrn_rate: {:.5f}, precision: {:.5f}".format(
      no_components, learning_rate, test_precision))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [ ]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

no_comp: 30, lrn_rate: 0.02251, precision: 0.14911
no_comp: 30, lrn_rate: 0.02564, precision: 0.15780
no_comp: 30, lrn_rate: 0.03518, precision: 0.17795
no_comp: 40, lrn_rate: 0.04393, precision: 0.19339
no_comp: 40, lrn_rate: 0.03950, precision: 0.18442
no_comp: 20, lrn_rate: 0.01010, precision: 0.10460
no_comp: 30, lrn_rate: 0.04648, precision: 0.19192
no_comp: 10, lrn_rate: 0.01688, precision: 0.10680
no_comp: 40, lrn_rate: 0.04810, precision: 0.19883
no_comp: 20, lrn_rate: 0.01336, precision: 0.11651
100%|██████████| 10/10 [00:18<00:00,  1.87s/it, best loss: -0.19882981479167938]


In [ ]:
no_components = 20
learning_rate = 0.04637

model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    random_state=0)

model.fit(interactions=train,
              item_features=item_features,
              sample_weight=train_weights,
              epochs=3,
              verbose=False)

In [ ]:
# Find Similar Items
item_biases, item_embeddings = model.get_item_representations(features=item_features)

def make_best_items_report(item_embeddings, advertiser_id, num_search_items=10):
    item_id = list(item_meta['advertiser_id']).index(advertiser_id) 

    # Cosine similarity
    scores = item_embeddings.dot(item_embeddings[item_id])  # (10000, )
    item_norms = np.linalg.norm(item_embeddings, axis=1)    # (10000, )
    item_norms[item_norms == 0] = 1e-10
    scores /= item_norms

    # best: score가 제일 높은 item의 id를 num_search_items 개 만큼 가져온다.
    best = np.argpartition(scores, -num_search_items)[-num_search_items:]
    similar_item_id_and_scores = sorted(zip(best, scores[best] / item_norms[item_id]),
                                        key=lambda x: -x[1])

    # Report를 작성할 pandas dataframe
    best_items = pd.DataFrame(columns=['advertiser_id', 'advertiser_favorite_count', 'advertiser_grade', 'score'])

    for similar_item_id, score in similar_item_id_and_scores:
        advertiser_id = item_meta['advertiser_id'][similar_item_id]
        advertiser_favorite_count = item_meta[item_meta['advertiser_id'] == advertiser_id].values[0][1]
        advertiser_grade = item_meta[item_meta['advertiser_id'] == advertiser_id].values[0][2]


        row = pd.Series([advertiser_id, advertiser_favorite_count, advertiser_grade, score], index=best_items.columns)
        best_items = best_items.append(row, ignore_index=True)

    return best_items


# book_id 2: Harry Potter and the Philosopher's Stone by J.K. Rowling, Mary GrandPré
# book_id 9: Angels & Demons by Dan Brown
report01 = make_best_items_report(item_embeddings, 12851816, 10)
report02 = make_best_items_report(item_embeddings, 74344938, 10)

In [ ]:
report01

,advertiser_id,advertiser_favorite_count,advertiser_grade,score
0,12851816.0,2131.0,5285.0,1.000000
1,87148.0,2183.0,12120.0,0.973576
2,74689400.0,29.0,511.0,0.929048
3,2790587.0,322.0,963.0,0.922558
4,12851816.0,2131.0,5285.0,0.920328
5,2498643.0,3580.0,8444.0,0.915516
6,3947717.0,7272.0,18135.0,0.888582
7,10197556.0,5649.0,26531.0,0.882459
8,315781.0,169.0,1572.0,0.879444
9,20993.0,541.0,486.0,0.877004


In [ ]:
item_meta[22:]

,advertiser_id,advertiser_favorite_count,advertiser_grade,advertiser_interest_count
22,87148,2183,12120,115655
23,12851816,2131,5285,4525
24,8105166,388,868,846
25,2701783,53,472,2649
26,12851816,2131,5285,4525
...,...,...,...,...
575187,87148,2183,12120,115655
575188,4776936,210,532,1053
575189,8602749,357,546,1507
575190,4565332,5816,26955,245194


In [ ]:
report02

,advertiser_id,advertiser_favorite_count,advertiser_grade,score
0,74344938.0,812.0,70.0,1.000000
1,3634682.0,376.0,2952.0,0.934166
2,4776936.0,210.0,532.0,0.917422
3,10197556.0,5649.0,26531.0,0.895218
4,1494515.0,7044.0,23571.0,0.875072
5,74344938.0,812.0,70.0,0.839226
6,10197556.0,5649.0,26531.0,0.771830
7,1266155.0,13291.0,20847.0,0.767604
8,12851816.0,2131.0,5285.0,0.739015
9,2487730.0,1899.0,7673.0,0.738002
